In [1]:
import tensorflow as tf
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import os

In [2]:
# MNIST 데이터셋 불러오기
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
# CNN 사용할 예정으로 차원수 추가
# label을 categorical로 활용

X_train = x_train.reshape(-1,28, 28, 1) 
X_test = x_test.reshape(-1,28,28,1)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [4]:
# train / val 분리
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
print(X_train.shape)

(48000, 28, 28, 1)


In [5]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(X, y, batch_size=trial.hyperparameters.Choice(
            'batch_size', [16, 32]), **fit_kwargs)


        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}

In [6]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape = X_train.shape[1:], name = 'inputs'))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
              model.add(tf.keras.layers.Conv2D(hp.Int(
                  'units_{i}'.format(i=i), min_value=32, max_value=128, step=5), (3,3),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    for i in range(hp.Int('n_connections', 1, 3)):
        model.add(tf.keras.layers.Dense(hp.Choice(f'n_nodes',
                                  values=[32,64,128, 256]), activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'outputs'))
    model.compile(optimizer = 'adam',loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [7]:
my_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_model,
    overwrite=True,
    project_name='my_keras_tuner')

# 해당 모델 학습시간은 약 10분정도 걸립니다!
my_keras_tuner.search(
    X_train, y_train, validation_data=(X_val, y_val), epochs=3)

Trial 10 Complete [00h 01m 01s]
accuracy: 0.9757500290870667

Best accuracy So Far: 0.9860833287239075
Total elapsed time: 00h 10m 26s


In [8]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
model = build_model(best_hps)
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_65 (Conv2D)          (None, 26, 26, 102)       1020      
                                                                 
 conv2d_66 (Conv2D)          (None, 24, 24, 42)        38598     
                                                                 
 conv2d_67 (Conv2D)          (None, 22, 22, 67)        25393     
                                                                 
 conv2d_68 (Conv2D)          (None, 20, 20, 37)        22348     
                                                                 
 conv2d_69 (Conv2D)          (None, 18, 18, 52)        17368     
                                                                 
 flatten_11 (Flatten)        (None, 16848)             0         
                                                                 
 dense_16 (Dense)            (None, 256)             

In [9]:
model.fit(X_train, y_train, batch_size=32, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 13s 8ms/step - loss: 0.1842 - accuracy: 0.9565
Epoch 2/5
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0507 - accuracy: 0.9844
Epoch 3/5
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0388 - accuracy: 0.9882
Epoch 4/5
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0286 - accuracy: 0.9906
Epoch 5/5
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0249 - accuracy: 0.9924


In [10]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 2s 6ms/step - loss: 0.0429 - accuracy: 0.9872


[0.042881835252046585, 0.9872000217437744]

In [12]:
save_path = 'C:/Users/Desk_Kang/Desktop/Aiffel/workplace/lib/mlops/best_model/model'
fname = os.path.join(save_path, 'model')
model.save(fname)

INFO:tensorflow:Assets written to: C:/Users/Desk_Kang/Desktop/Aiffel/workplace/lib/mlops/best_model/1\model\assets


INFO:tensorflow:Assets written to: C:/Users/Desk_Kang/Desktop/Aiffel/workplace/lib/mlops/best_model/1\model\assets


In [15]:
load_path = 'C:/Users/Desk_Kang/Desktop/Aiffel/workplace/lib/mlops/best_model/model/1'
best_model = tf.keras.models.load_model(load_path)

In [16]:
best_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_65 (Conv2D)          (None, 26, 26, 102)       1020      
                                                                 
 conv2d_66 (Conv2D)          (None, 24, 24, 42)        38598     
                                                                 
 conv2d_67 (Conv2D)          (None, 22, 22, 67)        25393     
                                                                 
 conv2d_68 (Conv2D)          (None, 20, 20, 37)        22348     
                                                                 
 conv2d_69 (Conv2D)          (None, 18, 18, 52)        17368     
                                                                 
 flatten_11 (Flatten)        (None, 16848)             0         
                                                                 
 dense_16 (Dense)            (None, 256)             

In [17]:
# tflite 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\DESK_K~1\AppData\Local\Temp\tmpft2026xu\assets


INFO:tensorflow:Assets written to: C:\Users\DESK_K~1\AppData\Local\Temp\tmpft2026xu\assets


In [18]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [19]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)

signatures = interpreter.get_signature_list()
print(signatures)

{'serving_default': {'inputs': ['inputs'], 'outputs': ['outputs']}}


In [20]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite